In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json

import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score

In [4]:
df = pd.read_excel("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/DataFrame.xlsx", index_col=0)
df.shape

(8281, 25)

In [49]:
# df.iloc[:, 15:25].head(1)

In [50]:
df.columns

Index(['pre_text', 'post_text', 'table_ori', 'table', 'qa', 'table_retrieved',
       'text_retrieved', 'table_retrieved_all', 'text_retrieved_all',
       'Company Name', 'Year of Publication', 'Page_Number', 'Operations',
       'Number of Operations', 'Supporting Facts', 'Number of Facts',
       'Rows from Text or Table', 'Text or Table', 'Model Input',
       'Number of Model Inputs', 'Top-N Facts', 'Number of Top-N Facts',
       'Question', 'Explanations', 'Answer'],
      dtype='object')

In [5]:
# Dropping the Unnecessary Columns

df = df.drop(columns =['table_retrieved', 'text_retrieved' , 'table_retrieved_all' , 'text_retrieved_all' , 
                       'Rows from Text or Table' , 'Model Input' , 'Top-N Facts' , 'Supporting Facts' , 'table_ori'])
df.shape

(8281, 16)

In [6]:
dh = pd.read_excel("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/PrePost_Column_Data.xlsx", index_col=0)
dh.head(1)

,pre_text,post_text
0,['interest rate to a variable interest rate ba...,['fair value of forward exchange contracts aft...


In [8]:
def preprocess(raw_text):
    # Removing special characters and digits
    sentence = re.sub("[^a-zA-Z0-9]", " ", str(raw_text))
    
    # change sentence to lower case
    sentence = sentence.lower()

    # tokenize into words
    tokens = sentence.split()
    return pd.Series([" ".join(tokens)])

In [9]:
dh['Pre_Text'] = dh['pre_text'].apply(lambda x: preprocess(x))
dh['Post_Text'] = dh['post_text'].apply(lambda x: preprocess(x))

In [10]:
dh = dh.drop(columns =['pre_text', 'post_text'])
# dh.head()

In [11]:
result = pd.concat([dh, df], axis=1)
result = result.drop(columns =['pre_text', 'post_text'])
result['Financial Context'] = result['Pre_Text'] + result['table'] + result['Post_Text']
# result.head()

In [3]:
# result.head(2)

In [58]:
result.shape

(8281, 17)

In [59]:
result = result.fillna(0)

In [60]:
# result.to_excel("Result.xlsx")

In [12]:
df_train = result.iloc[:6251]
# df_train.head()

In [13]:
df_test = result.iloc[6251:7398]
# df_test.head()

In [14]:
df_dev = result.iloc[7398:]
# df_dev.head()

In [15]:
def inputOutputSplit(df):
    Y = df["Answer"]
    X = df.drop(['Answer'], axis=1)
    return X,Y

In [16]:
X_train,Y_train = inputOutputSplit(df_train)

In [17]:
X_test,Y_test = inputOutputSplit(df_test)

In [18]:
X_dev,Y_dev = inputOutputSplit(df_dev)

# ChatGPT answer for df_train

In [82]:
def get_gpt_answer(fin_context, question):
    CONTENT = f"""Based on the provided Context, answer the Financial Question mentioned below. Generate only the numerical answer or percentage without any explanation.
    
    Context: {fin_context}
    Question: {question}
    
    Answer:
    """
    response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": CONTENT},
    ],
    temperature=0.6,
    )

    return response.choices[0].message.content.strip()
    

df_train['Generated Answer'] = df_train.apply(lambda x: get_gpt_answer(x["Financial Context"], x['Question']), axis=1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/peterchen/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_106892/1823884746.py", line 20, in <module>
    df_train['Generated Answer'] = df_train.apply(lambda x: get_gpt_answer(x["Financial Context"], x['Question']), axis=1)
  File "/home/peterchen/.local/lib/python3.8/site-packages/pandas/core/frame.py", line 7768, in apply
    return op.get_result()
  File "/home/peterchen/.local/lib/python3.8/site-packages/pandas/core/apply.py", line 185, in get_result
    return self.apply_standard()
  File "/home/peterchen/.local/lib/python3.8/site-packages/pandas/core/apply.py", line 276, in apply_standard
    results, res_index = self.apply_series_generator()
  File "/home/peterchen/.local/lib/python3.8/site-packages/pandas/core/apply.py", line 290, in apply_series_generator
    results[i] = self.f(v)
  File "/tmp/ipykernel_106

**(A) ChatGPT Models:**

In [69]:
!pip install -q openai
!pip install openai

In [1]:
import openai

In [28]:
context = df_train['Financial Context'][0]
question = df_train['Question'][0]
answer = df_train['Answer'][0]

In [29]:
answer

'380'

In [30]:
# Prompt 只有答案
CONTENT = f"""Based on the provided Context, consisting of the Pre-Text, Tabular Data, and the Post-Text Data, answer the 
Financial Question mentioned below. Use the given operations for computations as needed:

- For Unstructured Text Data: Add, Subtract, Multiply, Divide, Greater, Exponential (maximum 6 operations).
- For Structured Tabular Data: Table Maximum, Table Minimum, Table Sum, Table Average (maximum 4 operations).

Generate only the numerical answer. Do not include any explanation or steps.

Note: Carefully interpret numbers in the Structured Tabular Data, especially differentiating between decimal points (".") and commas (","). For example: 7,832 should be considered as 7832, not 7.832.

Context: {context}
Question: {question}

Answer:
"""

response_0 = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": CONTENT},
    ],
    temperature=0.5,
    # length_penalty=2,
    max_tokens=150
)

print(answer)
print(response_0.choices[0].message.content)

380
3.8 million


In [62]:
# Prompt 除了答案还有推理过程
CONTENT = f"""Based on the Context provided, consisting of the Pre-Text , Tabular Data , and the Post-Text Data, answer the 
Financial Question mentioned below. Since calculating Financial Answers involve Numerical Computations, there are a maximum of 
6 Mathematical Operations that should be used : Add , Subtract , Multiply , Divide , Greater , Exponential. And a maximum of 4 
Table Aggregation Operations that should be used : Table Maximum , Table Minimum , Table Sum , Table Average.
The 6 Mathematical Operations are for the Unstructured Text Data , and the 4 Table Aggregation Operations are for the Structured
Tabular Data.

Also, along with answering the Financial Question mentioned, provide a complete 
Explanation to understand how to reach at this Solution of the Financial Question generated. Means that in First Line, generate 
only the numerical answer, and then in the next line, generate the explanation.

Note: While generating answers, look carefully at the numbers which are present in the Structured Tabular Data, and do not get
confused between the symbols like decimal(".") and comma(","). For example: 7,832 should be considered as 7832 and not 7.832.
Be strict on noticing whether any number has decimal or comma, as it could change the answer a lot. 

Context : {context}
Question: {question}
 
Answer :
Explanation : 

"""
 
response_0 = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": CONTENT},
    ],
    temperature=0.6,
)

print(response_0.choices[0].message.content)

3.8 million

The interest expense in 2009 is mentioned as 3.8 million in the context provided. This expense would change if the LIBOR (London Interbank Offered Rate) changes by 100 basis points.


In [63]:
context = df_train['Financial Context'][2]

In [64]:
question = df_train['Question'][2]
question

'what was the total operating expenses in 2018 in millions'

In [65]:
CONTENT = f"""Based on the Context provided, consisting of the Pre-Text , Tabular Data , and the Post-Text Data, answer the 
Financial Question mentioned below. Since calculating Financial Answers involve Numerical Computations, there are a maximum of 
6 Mathematical Operations that should be used : Add , Subtract , Multiply , Divide , Greater , Exponential. And a maximum of 4 
Table Aggregation Operations that should be used : Table Maximum , Table Minimum , Table Sum , Table Average.
The 6 Mathematical Operations are for the Unstructured Text Data , and the 4 Table Aggregation Operations are for the Structured
Tabular Data.

Also, along with answering the Financial Question mentioned, provide a complete 
Explanation to understand how to reach at this Solution of the Financial Question generated. Means that in First Line, generate 
only the numerical answer, and then in the next line, generate the explanation.

Note: While generating answers, look carefully at the numbers which are present in the Structured Tabular Data, and do not get
confused between the symbols like decimal(".") and comma(","). For example: 7,832 should be considered as 7832 and not 7.832.
Be strict on noticing whether any number has decimal or comma, as it could change the answer a lot. 

Context : {context}
Question: {question}
 
Answer :
Explanation : 

"""
 
response_2 = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": CONTENT},
    ],
    temperature=0.3,
    length_penalty=1.2
)

print(response_2.choices[0].message.content)

$41.93 million
To calculate the total operating expenses in 2018, we need to use the table data provided. From the table, we see that the aircraft fuel expense for 2018 was $9896 million. Since the aircraft fuel expense was 23.6% of the total operating expenses, we can set up the equation: $9896 million = 23.6% of total operating expenses. Solving for the total operating expenses gives us $41.93 million. Therefore, the total operating expenses in 2018 were $41.93 million.


In [78]:
context = df_train['Financial Context'][1200]
question = df_train['Question'][1200]
answer = df_train['Answer'][1200]
print(context)
print(question)
print(answer)
CONTENT = f"""
You are a financial analysis assistant. Based on the provided Context, accurately answer the financial question mentioned below. Use the following operations for computations as needed:

For Unstructured Text Data, you can use the following mathematical operations: Add, Subtract, Multiply, Divide, Greater, Exponential (maximum 6 operations).
For Structured Tabular Data, you can use the following table aggregation operations: Table Maximum, Table Minimum, Table Sum, Table Average (maximum 4 operations).

Guidelines for Answer Generation:
1. Numerical Answer:
   - Generate only the number without any symbols like "$", words like "million", or commas ",".
   - For example, if the context says "$1,253.9 million", you should output "1253.9".
2. Yes/No Questions:
   - If the question requires a Yes/No answer, provide only 'yes' or 'no'.
3. Explanation:
   - Do not provide any explanation unless specifically asked in the question.

Important Notes:
- Pay special attention to numbers in the Structured Tabular Data. Distinguish between decimal points "." and commas ",". For example, interpret '7,832' as 7832, not 7.832.
- Always verify whether numbers have decimal points or commas to ensure accuracy in answers.

Context: {context}
Question: {question}

Answer:
"""

response_3 = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": CONTENT},
    ],
    temperature=0.6, 
)

print(response_3.choices[0].message.content)

note 6 2014mergers and acquisitions eldertrust merger on february 5 2004 the company consummated a merger transaction in an all cash transaction valued at 184 million the 201celdertrust transaction 201d the eldertrust transaction adds nine assisted living facilities one independent living facility five skilled nursing facilities two med ical office buildings and a financial office building the 201celdertrust properties 201d to the company 2019s portfolio the eldertrust properties are leased by the company to various operators under leases providing for aggregated annual cash base rent of approxi mately 16 2 million subject to escalation as provided in the leases the leases have remaining terms primarily ranging from four to 11 years at the closing of the eldertrust transaction the company also acquired all of the limited partnership units in eldertrust operating limited partnership 201cetop 201d directly from their owners at 12 50 per unit excluding 31455 class c units in etop which wi

**Out of OpenAI , Google PaLM and Google Gemini Pro Model, the best Question-Answer Responses are shown by OpenAI Model.**

In [30]:
import pandas as pd
from sklearn.metrics import accuracy_score

df_train = pd.read_csv("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/df_train_output.csv")
df_dev = pd.read_csv("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/df_dev_output.csv")
df_test = pd.read_csv("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/df_test_output.csv")
df = pd.concat([df_train, df_test, df_dev], axis=0)
df.shape

(8281, 18)

In [7]:
# 设置显示选项
# pd.set_option('display.max_rows', None)         # 显示所有行
# pd.set_option('display.max_columns', None)      # 显示所有列
# pd.set_option('display.max_colwidth', None)     # 不限制列宽
# pd.set_option('display.width', None)            # 自动调整宽度以适应内容
# pd.set_option('display.expand_frame_repr', False)  # 禁止折叠长列

pattern = r'\b(yes|no)\b'  # 匹配包含 'yes' 或 'no' 的字符串
# 标记包含 'yes' 或 'no' 的行
df['matches_pattern'] = df['Answer'].str.contains(pattern, case=False, regex=True)
# 筛选出包含 'yes' 或 'no' 的行
df[df['matches_pattern']][['Question', 'Answer', 'Generated Answer']]

NameError: name 'df' is not defined

In [40]:
df[['Question', 'Answer', 'Generated Answer']]

,Question,Answer,Generated Answer
0,what is the the interest expense in 2009?,380,3.8 million
1,"during the 2012 year , did the equity awards i...",0,3.3 million
2,what was the total operating expenses in 2018 ...,41932,"$41,905"
3,what percentage of total cash and investments ...,53%,53.69%
4,what is the growth rate in net revenue in 2008?,-3.2%,-3.23%
...,...,...,...
878,what portion of total future obligations is re...,45.6%,45.55%
879,what percentage of total future minimum lease ...,21%,21.47%
880,what was the percentage increase of total defe...,204%,204.29%
881,what would the amount accrued because of inter...,$ 2.33 million,$2.42 million


In [39]:
df[df['Answer'].str.contains("yes")]


,Pre_Text,Post_Text,table,qa,Company Name,Year of Publication,Page_Number,Operations,Number of Operations,Number of Facts,Text or Table,Number of Model Inputs,Number of Top-N Facts,Question,Explanations,Answer,Financial Context,Generated Answer,matches_pattern
47,jpmorgan chase co 2018 form 10 k 41 five year ...,december 31 in dollars,"[['december 31 ( in dollars )', '2013', '2014'...",{'question': 'did jpmorgan chase outperform th...,JPM,2018,73,greater,1,2,"table,table",3,1,did jpmorgan chase outperform the kbw bank index?,0,yes,jpmorgan chase co 2018 form 10 k 41 five year ...,"Yes, JPMorgan Chase outperformed the KBW Bank ...",False
86,page 92 of 98 other information required by it...,item 13 certain relationships and related tran...,"[['plan category', 'equity compensation plan i...","{'question': 'in the equity plans for 2006 , a...",BLL,2006,108,greater,1,1,table,3,2,"in the equity plans for 2006 , are there more ...",0,yes,page 92 of 98 other information required by it...,"Yes, there are more shares issued than remaini...",False
162,the goldman sachs group inc and subsidiaries m...,our average daily var decreased to 54 million ...,"[['$ in millions', 'year ended december 2017',...","{'question': 'in millions for 2016 , was the a...",GS,2017,104,greater,1,2,"table,table",3,1,"in millions for 2016 , was the average daily v...",0,yes,the goldman sachs group inc and subsidiaries m...,$20 million,False
273,apple inc 2018 form 10 k 20 company stock perf...,NaN,"[['', 'september2013', 'september2014', 'septe...",{'question': 'did apple outperform ( earn a gr...,AAPL,2018,23,greater,1,2,"table,table",3,1,did apple outperform ( earn a greater return )...,0,yes,apple inc 2018 form 10 k 20 company stock perf...,Apple outperformed the S&P Information Technol...,False
279,management 2019s discussion and analysis of fi...,segment results 20142009 compared to 2008 in 2...,"[['( dollars in millions )', 'years ended dece...",{'question': 'did consolidated net sales grow ...,MSI,2009,69,"multiply, multiply, subtract, divide",4,2,"table,text",3,1,did consolidated net sales grow from 2007 to 2...,in order to find out the consolidated net sale...,"yes , 38.2%",management 2019s discussion and analysis of fi...,-9%,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,b as of december 31 2014 the total amount auth...,equity compensation plan information informati...,"[['', 'december 312009', 'december 312010', 'd...","{'question': 'after 4 years , did the series c...",DISCA,2014,64,greater,1,2,"table,table",3,1,"after 4 years , did the series c outperform th...",0,yes,b as of december 31 2014 the total amount auth...,Series C did outperform the S&P 500 after 4 ye...,False
968,nearly all of the remaining increase in fuel e...,a effective january 1 2010 a new accounting st...,"[['millions', '2011', '2010', '2009'], ['cash ...","{'question': 'in 2012 , are the planned capita...",UNP,2011,24,greater,1,2,"text,text",3,1,"in 2012 , are the planned capital expenditures...",0,yes,nearly all of the remaining increase in fuel e...,$3.6 billion,False
247,are allocated using appropriate statistical ba...,13 financial instruments strategy and risk 201...,"[['millions', 'dec . 31 2011', 'dec . 31 2010'...","{'question': 'in 2011 , did the company distri...",UNP,2011,76,greater,1,2,"table,table",3,1,"in 2011 , did the company distribute more to s...","dividends to shareholders , interest to bondho...",yes,are allocated using appropriate statistical ba...,$284 million to shareholders vs $90 million to...,False
551,page 15 of 100 shareholder return performance ...,NaN,"[['', '12/31/05', '12/31/06', '12/31/07', '12/...",{'question': 'did the five year total return o...,BLL,2010,28,greater,1,2,"table,table",3,1,did the five year total return on ball corpora...,0,yes,page 15 of 100 shareholder return performance ...,"Yes, the five year total return on Ball Corpor...",False


In [35]:
# 去除没用的字符, 把数字和百分比转换为小数
def clean_and_convert(value):
    value = value.replace("$", "").replace(",", "")
    if 'million' in value:
        value = value.replace("million", "").strip()
        return float(value) * 1e6
    if "%" in value:
        value = value.replace("%", "").strip()
        return float(value) / 100
    # 转成浮点数
    return pd.to_numeric(value, errors='coerce')

# df[['Answer', 'Generated Answer']] = df[['Answer', 'Generated Answer']].applymap(clean_and_convert)
df["Answer"] = df["Answer"].apply(clean_and_convert)
df

ValueError: could not convert string to float: 'yes  38.2'

In [23]:
# 定义误差验证函数, 0.1%
def is_within_tolerance(expected, generated, tolerance_per = 0.1):
    if expected == 0:
        return generated == 0
    tolerance = tolerance_per / 100 * abs(expected)
    return abs(expected - generated) <= tolerance

# 验证误差验证函数
df['is_correct'] = df.apply(lambda row: is_within_tolerance(row['Answer'], row['Generated Answer']), axis=1)

df

,Pre_Text,Post_Text,table,qa,Company Name,Year of Publication,Page_Number,Operations,Number of Operations,Number of Facts,Text or Table,Number of Model Inputs,Number of Top-N Facts,Question,Explanations,Answer,Financial Context,Generated Answer,is_correct
0,interest rate to a variable interest rate base...,fair value of forward exchange contracts after...,"[['', 'october 31 2009', 'november 1 2008'], [...",{'question': 'what is the the interest expense...,ADI,2009,49,"divide, divide",2,1,text,3,2,what is the the interest expense in 2009?,0,380.00,interest rate to a variable interest rate base...,3.80,False
1,abiomed inc and subsidiaries notes to consolid...,the remaining unrecognized compensation expens...,"[['', 'number of shares ( in thousands )', 'we...","{'question': 'during the 2012 year , did the e...",ABMD,2012,75,"multiply, multiply, multiply, greater",4,2,"table,text",3,1,"during the 2012 year , did the equity awards i...",0,0.00,abiomed inc and subsidiaries notes to consolid...,3.30,False
2,the following table shows annual aircraft fuel...,as of december 31 2018 we did not have any fue...,"[['year', 'gallons', 'average priceper gallon'...",{'question': 'what was the total operating exp...,AAL,2018,13,divide,1,1,table,3,2,what was the total operating expenses in 2018 ...,the total operating expenses is obtained by di...,41932.00,the following table shows annual aircraft fuel...,NaN,False
3,the fair value of our grants receivable is det...,in the third quarter of 2013 we sold our share...,"[['( in millions )', 'dec 282013', 'dec 292012...",{'question': 'what percentage of total cash an...,INTC,2013,71,divide,1,2,"table,table",3,1,what percentage of total cash and investments ...,0,NaN,the fair value of our grants receivable is det...,NaN,False
4,entergy louisiana llc management s financial d...,the retail electric price variance is primaril...,"[['', 'amount ( in millions )'], ['2007 net re...",{'question': 'what is the growth rate in net r...,ETR,2008,313,"subtract, divide",2,2,"table,table",3,1,what is the growth rate in net revenue in 2008?,0,NaN,entergy louisiana llc management s financial d...,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,contractual obligations and commercial commitm...,we have no long term debt capital leases or ma...,"[['contractual obligations', 'payments due by ...",{'question': 'what portion of total future obl...,ABMD,2007,52,divide,1,2,"table,table",3,1,what portion of total future obligations is re...,0,NaN,contractual obligations and commercial commitm...,NaN,False
879,abiomed inc and subsidiaries notes to consolid...,from time to time the company is involved in l...,"[['fiscal year ending march 31,', 'operating l...",{'question': 'what percentage of total future ...,ABMD,2006,75,divide,1,2,"table,table",3,1,what percentage of total future minimum lease ...,0,NaN,abiomed inc and subsidiaries notes to consolid...,NaN,False
880,contingent consideration of up to 13 8 million...,the fair values of these investments are based...,"[['', '2011', '2010'], ['money market funds', ...",{'question': 'what was the percentage increase...,ADI,2011,81,"subtract, divide",2,2,"table,text",3,1,what was the percentage increase of total defe...,to find the percentage increase one must subtr...,NaN,contingent consideration of up to 13 8 million...,NaN,False
881,we hold an interest rate swap agreement to hed...,fair value of forward exchange contracts after...,"[['', 'october 29 2011', 'october 30 2010'], [...",{'question': 'what would the amount accrued be...,ADI,2011,50,multiply,1,1,text,3,2,what would the amount accrued because of inter...,to find the amount of money that would come fr...,2.33,we hold an interest rate swap agreement to hed...,2.42,False


# 验证 gpt 生成答案的正确率

In [72]:
import pandas as pd

df = pd.read_csv("/home/peterchen/Model/Data_Science_Projects/FINQA Project - A Dataset of Numerical Reasoning over Financial Data/result_output.csv")
df.tail()

,Pre_Text,Post_Text,table,qa,Company Name,Year of Publication,Page_Number,Operations,Number of Operations,Number of Facts,Text or Table,Number of Model Inputs,Number of Top-N Facts,Question,Explanations,Answer,Financial Context,Generated Answer
8276,contractual obligations and commercial commitm...,we have no long term debt capital leases or ma...,"[['contractual obligations', 'payments due by ...",{'question': 'what portion of total future obl...,ABMD,2007,52,divide,1,2,"table,table",3,1,what portion of total future obligations is re...,0,45.6%,contractual obligations and commercial commitm...,47.8
8277,abiomed inc and subsidiaries notes to consolid...,from time to time the company is involved in l...,"[['fiscal year ending march 31,', 'operating l...",{'question': 'what percentage of total future ...,ABMD,2006,75,divide,1,2,"table,table",3,1,what percentage of total future minimum lease ...,0,21%,abiomed inc and subsidiaries notes to consolid...,21.48
8278,contingent consideration of up to 13 8 million...,the fair values of these investments are based...,"[['', '2011', '2010'], ['money market funds', ...",{'question': 'what was the percentage increase...,ADI,2011,81,"subtract, divide",2,2,"table,text",3,1,what was the percentage increase of total defe...,to find the percentage increase one must subtr...,204%,contingent consideration of up to 13 8 million...,203.77
8279,we hold an interest rate swap agreement to hed...,fair value of forward exchange contracts after...,"[['', 'october 29 2011', 'october 30 2010'], [...",{'question': 'what would the amount accrued be...,ADI,2011,50,multiply,1,1,text,3,2,what would the amount accrued because of inter...,to find the amount of money that would come fr...,$ 2.33 million,we hold an interest rate swap agreement to hed...,1.61
8280,debt maturities 2013 the following table prese...,at december 31 2007 we reclassified as long te...,"[['2008', '$ 689'], ['2009', '542'], ['2010', ...",{'question': 'what percentage of total debt is...,UNP,2007,65,divide,1,2,"table,table",3,1,what percentage of total debt is due after 2012?,0,61%,debt maturities 2013 the following table prese...,61.4


In [73]:
print("Original rows: ", len(df))
# 找到重复行
dup_rows = df[df.duplicated()] 
# 获取重复的行号
dup_idx = df[df.duplicated()].index.tolist()
# print("Duplicated rows: ", dup_rows)
# print("Duplicated indices: ", dup_idx)
print("Duplicated numbers: ", len(dup_rows))
df_drop_dup = df.drop_duplicates()
print("After drop: ", len(df_drop_dup))

Original rows:  8281
Duplicated numbers:  21
After drop:  8260


In [ ]:
""""" "pre_text": ["the texts before the table", "..."];
"post_text": ["the text after the table", "..."];
"filename": "unique example id";
"table_ori": [
    [
        "",
        "October 31, 2009",
        "November 1, 2008"
    ],
    [
        "Fair value of forward exchange contracts asset (liability)",
        "$6,427",
        "$(23,158)"
    ], ...];
"table": Convert all characters to lowercase;

"qa": {
    "question":
    "answer": 
    "explanation":
    "ann_table_rows":
    "ann_text_rows":
    "steps": [
        {
            "op": "divide1-1",
            "arg1": "100",
            "arg2": "100",
            "res": "1%"
        }]
    "program": "divide(100, 100), divide(3.8, #0)";
    "gold_inds": the gold supporting facts;
    "exe_ans": the gold execution result;
    "tfidftopn": use tf-idf to search for the relevant texts;
    "program_re": the reasoning program in nested format;
    "model_input":
},

"id": same as filename
"table_retrieved":
"text_retrieved":
"table_retrieved_all":
"text_retrieved_all": """""

In [51]:
# 查找对应值所在的行
target_value = "why is the information relative to 2012 costs incorrect and what would the correct information be?"
df[df["Question"] == target_value].index.tolist()

[2825, 3941]

In [75]:
pd.reset_option("display.max_colwidth")
# pd.set_option("display.max_colwidth", None)
# df.iloc[[2825, 3941]]
df.head()

,Pre_Text,Post_Text,table,qa,Company Name,Year of Publication,Page_Number,Operations,Number of Operations,Number of Facts,Text or Table,Number of Model Inputs,Number of Top-N Facts,Question,Explanations,Answer,Financial Context,Generated Answer
0,interest rate to a variable interest rate base...,fair value of forward exchange contracts after...,"[['', 'october 31 2009', 'november 1 2008'], [...",{'question': 'what is the the interest expense...,ADI,2009,49,"divide, divide",2,1,text,3,2,what is the the interest expense in 2009?,0,380.0,interest rate to a variable interest rate base...,3.8
1,abiomed inc and subsidiaries notes to consolid...,the remaining unrecognized compensation expens...,"[['', 'number of shares ( in thousands )', 'we...","{'question': 'during the 2012 year , did the e...",ABMD,2012,75,"multiply, multiply, multiply, greater",4,2,"table,text",3,1,"during the 2012 year , did the equity awards i...",0,0.0,abiomed inc and subsidiaries notes to consolid...,no
2,the following table shows annual aircraft fuel...,as of december 31 2018 we did not have any fue...,"[['year', 'gallons', 'average priceper gallon'...",{'question': 'what was the total operating exp...,AAL,2018,13,divide,1,1,table,3,2,what was the total operating expenses in 2018 ...,the total operating expenses is obtained by di...,41932.0,the following table shows annual aircraft fuel...,41.89
3,the fair value of our grants receivable is det...,in the third quarter of 2013 we sold our share...,"[['( in millions )', 'dec 282013', 'dec 292012...",{'question': 'what percentage of total cash an...,INTC,2013,71,divide,1,2,"table,table",3,1,what percentage of total cash and investments ...,0,53.0,the fair value of our grants receivable is det...,53.33
4,entergy louisiana llc management s financial d...,the retail electric price variance is primaril...,"[['', 'amount ( in millions )'], ['2007 net re...",{'question': 'what is the growth rate in net r...,ETR,2008,313,"subtract, divide",2,2,"table,table",3,1,what is the growth rate in net revenue in 2008?,0,-3.2,entergy louisiana llc management s financial d...,-3.23


In [50]:
# pd.set_option("display.max_colwidth", None)
# df[["Question", "Explanations", "Answer", "Generated Answer"]].iloc[2600:2620]


[2825, 3941]

In [80]:
answer_col = "Answer"
gen_col = "Generated Answer"
df[answer_col] = df[answer_col].apply(process_answer)
df[gen_col] = df[gen_col].apply(process_answer)

In [85]:
pd.set_option("max_colwidth", None)
mask = df["Answer"].str.len() > 10
# df[mask][["Question", "Answer", "Generated Answer"]]

In [89]:

# df.iloc[500:510]
df_filtered = df[~df[answer_col].apply(lambda x: isinstance(x, str) and any(char.isalpha() for char in x.split()) and x.strip().lower() not in ['yes','no'])]
# mask = df_filtered["Answer"].str.len()>10
# df_filtered[mask][["Question", "Answer", "Generated Answer"]]
pd.reset_option("max_colwidth")
df_filtered

,Pre_Text,Post_Text,table,qa,Company Name,Year of Publication,Page_Number,Operations,Number of Operations,Number of Facts,Text or Table,Number of Model Inputs,Number of Top-N Facts,Question,Explanations,Answer,Financial Context,Generated Answer
0,interest rate to a variable interest rate base...,fair value of forward exchange contracts after...,"[['', 'october 31 2009', 'november 1 2008'], [...",{'question': 'what is the the interest expense...,ADI,2009,49,"divide, divide",2,1,text,3,2,what is the the interest expense in 2009?,0,380.0,interest rate to a variable interest rate base...,3.8
1,abiomed inc and subsidiaries notes to consolid...,the remaining unrecognized compensation expens...,"[['', 'number of shares ( in thousands )', 'we...","{'question': 'during the 2012 year , did the e...",ABMD,2012,75,"multiply, multiply, multiply, greater",4,2,"table,text",3,1,"during the 2012 year , did the equity awards i...",0,0.0,abiomed inc and subsidiaries notes to consolid...,no
2,the following table shows annual aircraft fuel...,as of december 31 2018 we did not have any fue...,"[['year', 'gallons', 'average priceper gallon'...",{'question': 'what was the total operating exp...,AAL,2018,13,divide,1,1,table,3,2,what was the total operating expenses in 2018 ...,the total operating expenses is obtained by di...,41932.0,the following table shows annual aircraft fuel...,41.89
3,the fair value of our grants receivable is det...,in the third quarter of 2013 we sold our share...,"[['( in millions )', 'dec 282013', 'dec 292012...",{'question': 'what percentage of total cash an...,INTC,2013,71,divide,1,2,"table,table",3,1,what percentage of total cash and investments ...,0,53.0,the fair value of our grants receivable is det...,53.33
4,entergy louisiana llc management s financial d...,the retail electric price variance is primaril...,"[['', 'amount ( in millions )'], ['2007 net re...",{'question': 'what is the growth rate in net r...,ETR,2008,313,"subtract, divide",2,2,"table,table",3,1,what is the growth rate in net revenue in 2008?,0,-3.2,entergy louisiana llc management s financial d...,-3.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,contractual obligations and commercial commitm...,we have no long term debt capital leases or ma...,"[['contractual obligations', 'payments due by ...",{'question': 'what portion of total future obl...,ABMD,2007,52,divide,1,2,"table,table",3,1,what portion of total future obligations is re...,0,45.6,contractual obligations and commercial commitm...,47.8
8277,abiomed inc and subsidiaries notes to consolid...,from time to time the company is involved in l...,"[['fiscal year ending march 31,', 'operating l...",{'question': 'what percentage of total future ...,ABMD,2006,75,divide,1,2,"table,table",3,1,what percentage of total future minimum lease ...,0,21.0,abiomed inc and subsidiaries notes to consolid...,21.48
8278,contingent consideration of up to 13 8 million...,the fair values of these investments are based...,"[['', '2011', '2010'], ['money market funds', ...",{'question': 'what was the percentage increase...,ADI,2011,81,"subtract, divide",2,2,"table,text",3,1,what was the percentage increase of total defe...,to find the percentage increase one must subtr...,204.0,contingent consideration of up to 13 8 million...,203.77
8279,we hold an interest rate swap agreement to hed...,fair value of forward exchange contracts after...,"[['', 'october 29 2011', 'october 30 2010'], [...",{'question': 'what would the amount accrued be...,ADI,2011,50,multiply,1,1,text,3,2,what would the amount accrued because of inter...,to find the amount of money that would come fr...,2.33,we hold an interest rate swap agreement to hed...,1.61


In [92]:
import re
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

# 预处理
def process_answer(answer):
    if isinstance(answer, str):
        answer = re.sub(r"[,\%\$]", "", answer)
        answer = re.sub(r"million", "", answer, flags=re.IGNORECASE)
        try:
            return float(answer)
        except ValueError:
            return answer.strip().lower()
    elif isinstance(answer, (int, float)):
        return float(answer)
    else:
        return None

# 比较两个值是否相等, 允许一定的误差
def compare_values(val1, val2, tolerance=1e-2):
    if isinstance(val1, (int, float)) and isinstance(val2, (int, float)):
        return np.isclose(val1, val2, atol=tolerance)
    return val1 == val2

# 验证生成的但的准确率
def validate_answer(df, answer_col, gen_col, tolerance=1e-2):
    # 处理答案
    df[answer_col] = df[answer_col].apply(process_answer)
    df[gen_col] = df[gen_col].apply(process_answer)
    # 过滤掉生成答案种包含句子的行
    df_filtered = df[~df[answer_col].apply(lambda x: isinstance(x, str) and any(char.isalpha() for char in x.split()) and x.strip().lower() not in ['yes','no'])]
    # 比较答案
    df_filtered['is_correct'] = df_filtered.apply(
        lambda row: compare_values(row[answer_col], row[gen_col], tolerance=tolerance), axis=1
    )
    # 计算正确率
    acc = accuracy_score(df_filtered['is_correct'], [True] * len(df_filtered))
    return acc

accuracy = validate_answer(df, 'Answer', 'Generated Answer')
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 21.14%


/tmp/ipykernel_517804/4010694681.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['is_correct'] = df_filtered.apply(
